# Real time detection

In [ ]:
import cv2	
import numpy as np
import os

# Load Yolo
net = cv2.dnn.readNet("yolov3_training_last.weights", "yolov3_testing.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
classes = ["mask", "no_mask"]
colors = [(0, 0, 255), (0, 255, 0)]

def draw_predictions(frame, class_ids, scores, xmin, ymin, xmax, ymax):
	for i in range(len(class_ids)):
		label = str(classes[class_ids[i]])
		color = colors[class_ids[i]]
		cv2.rectangle(frame, (xmin[i], ymin[i]), (xmax[i], ymax[i]), color, 2)
		cv2.putText(frame, label, (xmin[i], ymin[i]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
	return frame
cap = cv2.VideoCapture(0)
while True:
	_, frame = cap.read()
	height, width, channels = frame.shape
	# Detecting objects
	blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
	net.setInput(blob)
	outs = net.forward(output_layers)
	# Showing informations on the screen
	class_ids = []
	confidences = []
	xmin = []
	ymin = []
	xmax = []
	ymax = []
	for out in outs:
		for detection in out:
			scores = detection[5:]
			class_id = np.argmax(scores)
			confidence = scores[class_id]
			if confidence > 0.3:
				# Object detected
				center_x = int(detection[0]*width)
				center_y = int(detection[1]*height)
				w = int(detection[2]*width)
				h = int(detection[3]*height)
				# Rectangle coordinates
				x = int(center_x - w/2)
				y = int(center_y - h/2)
				xmin.append(x)
				ymin.append(y)
				xmax.append(x+w)
				ymax.append(y+h)
				confidences.append(float(confidence))
				class_ids.append(class_id)
	indices = cv2.dnn.NMSBoxes(xmin, ymin, xmax, ymax, 0.4, 0.3)
	for i in indices:
		i = i[0]
		box = [xmin[i], ymin[i], xmax[i], ymax[i]]
		frame = draw_predictions(frame, class_ids, confidences, xmin, ymin, xmax, ymax)
	cv2.imshow("Image", frame)
	key = cv2.waitKey(1)
	if key == 27:
		break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@0.052] global loadsave.cpp:248 findDecoder imread_('test.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/caffe/caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "MobileNetSSD_deploy.prototxt.txt" in function 'ReadProtoFromTextFile'
